#### Chroma
Chroma is a AI-native open-source vector database focused on developer productivity and happiness. Chroma is licensed under Apache 2.0.

https://python.langchain.com/v0.2/docs/integrations/vectorstores/

In [2]:
## building a sample vectordb

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings

from langchain_chroma import Chroma

In [4]:
## loading text 
loader = TextLoader("data_ingestion/speech.txt")
data = loader.load()
data

[Document(metadata={'source': 'data_ingestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right 

In [5]:
## split 
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 0)
splits = text_splitter.split_documents(data)

In [6]:
embeddings = OllamaEmbeddings(model = "gemma2:2b")
vectordb = Chroma.from_documents(documents=splits,embedding=embeddings)
vectordb

C:\Users\thaku\AppData\Local\Temp\ipykernel_25024\3015166230.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model = "gemma2:2b")


In [7]:
## query database
query = "What does the speaker believe is the main reason United states should end war"
## query that we want response for based on the document provided 
docs = vectordb.similarity_search(query)

In [9]:
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy,'

In [11]:
## saving to the disk
vectordb = Chroma.from_documents(splits ,embeddings,persist_directory="./chroma_db")

## internally creates sqlite db
## every vector stored inside it, from our docuemnt using embeddings 

In [12]:
## Loading from disk
db2 = Chroma(persist_directory= "./chroma_db", embedding_function= embeddings)

In [16]:
docs = db2.similarity_search(query)

In [17]:
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy,'

In [19]:
docs = db2.similarity_search_with_score(query)

In [20]:
docs

[(Document(metadata={'source': 'data_ingestion/speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy,'),
  12280.808955162482),
 (Document(metadata={'source': 'data_ingestion/speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all co

In [21]:
## Manhatan dist used

In [22]:
retreiver = vectordb.as_retriever()
retreiver.invoke(query)[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy,'

: 